# Inputs - representación de las palabras

Lo primero que tenemos que ver es cómo le introducimos una frase al transformer

<div style="text-align:center;">
  <img src="Imagenes/transformer_architecture_model_inputs.png" alt="Encoder inputs" style="width:425px;height:626px;">
</div>

# Representación de las palabras

Como hemos visto en el tema de redes convolucionales, las imágenes se representan como la cantidad de rojo, verde y azul de cada pixel (RGB). Esa cantidad es un número que varía de entre 0 y 255. Es decir, para representar una imagen necesitamos hacerlo mediante números.

Con el lenguaje pasa igual, para poder procesarlo y realizar predicciones o generar texto, necesitamos poder representarlo mediante números. Para ello se utilizan los `token`s

## Tokens

### Bert tokenizer

Para explicar qué son los `token`s, primero veámoslo con un ejemplo práctico, vamos a usar el tokenizador de BERT

BERT es un modelo grande de lenguaje creado por Google. Para usar su tokenizador vamos a usar la librería de huggingface

La empresa [huggingface](https://huggingface.co) se hizo muy popular gracias a su librería [transformers](https://huggingface.co/docs/transformers), ya que hizo que fuese muy sencillo usarlos gracias a su API. Ahora no solo podemos usar transformers, sino casi cualquier modelo, ya que se autodefinen como el GitHub del deep learning, así que casi todos los nuevos modelos se publican también en huggingface, donde los puedes usar con su librería, ver su documentación e incluso probar online gracias a sus `spaces`

En este curso no vamos a explicar hugginface, pero es bueno que la conozcas, ya que es un muy buen recurso

Primero vamos a importar las librerías necesarias

In [1]:
from transformers import BertTokenizer, BertModel
import torch

Cargamos el modelo BERT

In [3]:
model = BertModel.from_pretrained('bert-base-multilingual-cased')

Seguramente obtendrás un warning como yo, ya que estamos importando los pesos de `bert-base-multilingual-cased` que fue entrenado con más capas de las que aparecen en el modelo `BertModel`. Si fuésemos a usar el modelo para lo que fue entrenado y dio como resultado los pesos `bert-base-multilingual-cased` deberíamos importar las capas extra, pero como no es nuestro caso podemos ignorar el warning

Ahora cargamos su tokenizador

In [5]:
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

Obtenemos los `token`s de la palabra `hello`

In [7]:
input_ids = tokenizer.encode("hello")
input_ids

[101, 61694, 10133, 102]

Vamos a analizar los `ids` o `token`s.

Por cómo está implementado BERT siempre espera que cada frase empieze con el token `[CLS]` (`101`) y que termine con el token `[SEP]` (`102`), por lo que el tokenizador de BERT los añade automáticamente

Por otro lado los tokens `61694` y `10133` son los tokens generados por dividir la palabra `hello` en dos subpalabras, que son

In [8]:
tokens = tokenizer.convert_ids_to_tokens(input_ids)
tokens

['[CLS]', 'hell', '##o', '[SEP]']

BERT ha dividido la palabra `hello` en `hell` y `##o`.

Cuando se crea el vocabulario que se quiere que tenga nuestro modelo, se hacen descomposiciones de palabras en otras más sencillas, a estas se les llaman `token`s

De modo que no habrá un token `perro` y otro `perra` sino que habrá un token `perr`, otro `##o` y otro `##a` (las dobles almohadillas indican que son el final de una palabra).

Puedes pensar, pero `perro` y `perra` son dos tokens y `perr`, `##o` y `##a` son tres tokens, y es verdad, pero ahora podemos añadir el token `herman` y juntarlo con los tokens ya creados `##o` y `##a`, y añadir el token `cuñad` y volver a juntarlo con los tokens ya creados `##o` y `##a`. De esta manera hemos pasado de tener 6 tokens `perro`, `perra`, `hermano`, `hermana`, `cuñado` y `cuñada` a solo 5 con `perr`, `herman`, `cuñad`, `##o` y `##a`. Hemos ahorrado un token, y a medida que añadamos más palabras terminadas en `o` o `a` seguiremos reduciendo el número de tokens.

Esta técnica y otras de dividir las palabras en otras más sencillas se llama tokenización, lo que produce un vocabulario más pequeño y así poder usar modelos más pequeños

### Entrenamiento de nuestro propio tokenizer

Hemos usado el tokenizador de BERT, pero este fue entrenado con un corpus sobre todo de palabras en inglés, por lo que si quisiésemos usar un transformer en otro idioma estaría bien entrenar un tokenizador con un corpus de ese idioma para que las palabras se dividan en `token`s de manera más adecuada

Hugging Face tiene la librería [tokenizers](https://huggingface.co/docs/tokenizers/index) que nos ayudará a entrenar nuestro propio tokenizador, y la librería [datasets](https://huggingface.co/docs/datasets/index) que nos ayudará a descargar corpus de texto para entrenar nuestro tokenizador

Como hemos dicho antes, este no es un curso de Hugging Face, por lo que simplemente usaremos el código para ver cómo se entrena un tokenizador

Primero descargamos el dataset [wikitext](https://huggingface.co/datasets/wikitext), que es una colección de más de 100 millones de tokens extraídos del conjunto de artículos verificados Buenos y Destacados de Wikipedia

In [1]:
import datasets

dataset = datasets.load_dataset("wikitext", "wikitext-103-raw-v1", split="train+test+validation")

Vamos a ver un poco cómo es el dataset

In [2]:
dataset

Dataset({
    features: ['text'],
    num_rows: 1809468
})

Vemos que es una especie de diccionario, así que vemos la clave `text`

In [3]:
type(dataset['text'])

list

Vemos que es una lista, así que vamos a ver un elemento

In [4]:
idx = 100
type(dataset['text'][idx]), dataset['text'][idx]

(str, ' 96 ammunition packing boxes \n')

Vemos que cada elemento es un string

Pasamos ahora a crear un nuevo `tokenizer`

In [5]:
from tokenizers import Tokenizer
from tokenizers.models import WordPiece
from tokenizers import normalizers
from tokenizers.normalizers import NFD, Lowercase, StripAccents
from tokenizers.pre_tokenizers import Whitespace

tokenizer = Tokenizer(WordPiece(unk_token="[UNK]"))
tokenizer.normalizer = normalizers.Sequence([NFD(), Lowercase(), StripAccents()])
tokenizer.pre_tokenizer = Whitespace()

Creamos un `trainer` que será el encargado de entrenar el `tokenizer`. Lo creamos con un `vocab_size`, el vocabulario, es decir, el número de `token`s distintos que queremos que tenga nuestro `tokenizer`

In [6]:
from tokenizers.trainers import WordPieceTrainer

trainer = WordPieceTrainer(vocab_size=100, special_tokens=["[UNK]", "[CLS]", "[SEP]", "[PAD]", "[MASK]"])

Creamos una función que iterará sobre el dataset y le pasará cada string al `trainer`

In [9]:
from tqdm import tqdm

def batch_iterator(batch_size=1000):
    iterator = tqdm(range(0, len(dataset["text"]), batch_size))
    for i in iterator:
        yield dataset[i : i + batch_size]["text"]

Entrenamos el `tokenizer`

In [10]:
tokenizer.train_from_iterator(batch_iterator(), trainer=trainer, length=len(dataset))

100%|██████████| 1810/1810 [00:13<00:00, 129.36it/s]


Ahora que tenemos el tokenizador entrenado y personalizado para nuestro corpus